In [1]:
import os
import kagglehub
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Download the dataset
path = kagglehub.dataset_download("jagdishchakole1/mango-data")
print("Path to dataset files:", path)

# Define data directories
train_dir = os.path.join(path, "Mango_leaf_disease1/train")
val_dir = os.path.join(path, "Mango_leaf_disease1/val")
test_dir = os.path.join(path, "Mango_leaf_disease1/test")

100%|██████████| 104M/104M [00:00<00:00, 133MB/s] 

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/jagdishchakole1/mango-data/versions/1


In [2]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=25,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Just rescaling for validation and test sets
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Set up the generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(299, 299),  # InceptionV3 requires 299x299 input
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)


Found 2800 images belonging to 8 classes.
Found 800 images belonging to 8 classes.
Found 400 images belonging to 8 classes.


In [3]:
# Get the number of classes
num_classes = len(train_generator.class_indices)
print(f"Number of classes: {num_classes}")
print("Class mapping:", train_generator.class_indices)


Number of classes: 8
Class mapping: {'Anthracnose': 0, 'Bacterial Canker': 1, 'Cutting Weevil': 2, 'Die Back': 3, 'Gall Midge': 4, 'Healthy': 5, 'Powdery Mildew': 6, 'Sooty Mould': 7}


In [4]:
# Create the base model from the pre-trained InceptionV3
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)  # Add dropout to prevent overfitting
predictions = Dense(num_classes, activation='softmax')(x)

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [5]:
# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [6]:
# Add callbacks for better training
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=3,
    min_lr=1e-6
)


In [7]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=15,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size,
    callbacks=[early_stopping, reduce_lr]
)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc:.4f}")

# Save the model
model.save('mango_disease_inception_model.h5')

Epoch 1/15


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


87/87 ━━━━━━━━━━━━━━━━━━━━ 112s 875ms/step - accuracy: 0.5977 - loss: 1.2596 - val_accuracy: 0.9175 - val_loss: 0.3209 - learning_rate: 1.0000e-04
Epoch 2/15
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 629us/step - accuracy: 0.8750 - loss: 0.3600 - learning_rate: 1.0000e-04
Epoch 3/15


/usr/lib/python3.11/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)
/usr/local/lib/python3.11/dist-packages/keras/src/callbacks/early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)
/usr/local/lib/python3.11/dist-packages/keras/src/callbacks/callback_list.py:96: UserWarning: Learning rate reduction is conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss,learning_rate.
  callback.on_epoch_end(epoch, logs)


87/87 ━━━━━━━━━━━━━━━━━━━━ 63s 678ms/step - accuracy: 0.9171 - loss: 0.2988 - val_accuracy: 0.9388 - val_loss: 0.2043 - learning_rate: 1.0000e-04
Epoch 4/15
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 267us/step - accuracy: 0.9688 - loss: 0.1351 - learning_rate: 1.0000e-04
Epoch 5/15
87/87 ━━━━━━━━━━━━━━━━━━━━ 64s 692ms/step - accuracy: 0.9367 - loss: 0.2165 - val_accuracy: 0.9425 - val_loss: 0.1691 - learning_rate: 1.0000e-04
Epoch 6/15
87/87 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - accuracy: 0.9375 - loss: 0.1794 - learning_rate: 1.0000e-04
Epoch 7/15
87/87 ━━━━━━━━━━━━━━━━━━━━ 79s 687ms/step - accuracy: 0.9521 - loss: 0.1616 - val_accuracy: 0.9513 - val_loss: 0.1405 - learning_rate: 1.0000e-04
Epoch 8/15
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 304us/step - accuracy: 0.9688 - loss: 0.1284 - learning_rate: 1.0000e-04
Epoch 9/15
87/87 ━━━━━━━━━━━━━━━━━━━━ 64s 685ms/step - accuracy: 0.9596 - loss: 0.1446 - val_accuracy: 0.9563 - val_loss: 0.1237 - learning_rate: 1.0000e-04
Epoch 10/15
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 

Test accuracy: 0.9650
